In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["ampform[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Formulate amplitude model

## Generate transitions

In {doc}`qrules:usage/reaction`, we used {func}`~qrules.generate_transitions` to create a list of allowed {class}`~qrules.transition.StateTransition`s for a specific decay channel:

In [ ]:
import qrules

reaction = qrules.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)(980)", "f(0)(1500)"],
    allowed_interaction_types=["strong", "EM"],
    formalism="canonical-helicity",
)

In [ ]:
import graphviz

dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

## Build SymPy expression

We can now use the {class}`~qrules.transition.ReactionInfo` to formulate an amplitude model. The type of this amplitude model is dependent on the {attr}`~qrules.transition.ReactionInfo.formalism`. The function {func}`.get_builder` helps to get the correct amplitude builder class for this {attr}`~qrules.transition.ReactionInfo.formalism`:

In [ ]:
from ampform import get_builder

model_builder = get_builder(reaction)
type(model_builder)

If we now use the {meth}`.HelicityAmplitudeBuilder.formulate` method of this builder, we get an amplitude model without any dynamics:

In [ ]:
model_no_dynamics = model_builder.formulate()

The {attr}`.HelicityModel.expression` is just a {class}`sympy.Expr <sympy.core.expr.Expr>`, which we can pull apart by using its {attr}`~sympy.core.basic.Basic.args` (see {doc}`sympy:tutorial/manipulation`):

In [ ]:
intensities = model_no_dynamics.expression.args
intensity_1 = intensities[0]
base, power = intensity_1.args
abs_arg = base.args[0]
amplitudes = abs_arg.args
amplitudes[0]

To set dynamics for specific resonances, use {meth}`.set_dynamics` on the same {class}`.HelicityAmplitudeBuilder` instance.  You can set the dynamics to be any kind of {class}`~sympy.core.expr.Expr`, as long as you keep track of which {class}`~sympy.core.symbol.Symbol` names you use. AmpForm does provide a few common {mod}`.dynamics` functions however, which can be constructed as {class}`~sympy.core.expr.Expr` with the correct {class}`~sympy.core.symbol.Symbol` names using {meth}`.set_dynamics`. This function takes specific {mod}`.dynamics.builder` functions, such as {func}`.create_relativistic_breit_wigner`, which would create a {func}`.relativistic_breit_wigner` for a specific resonance. Here's an example for a relativistic Breit-Wigner _with form factor_ for the intermediate resonances and use a Blatt-Weisskopf barrier factor for the production decay:

In [ ]:
from ampform.dynamics.builder import (
    create_non_dynamic_with_ff,
    create_relativistic_breit_wigner_with_ff,
)

initial_state_particle = reaction.initial_state[-1]
model_builder.set_dynamics(initial_state_particle, create_non_dynamic_with_ff)
for name in reaction.get_intermediate_particles().names:
    model_builder.set_dynamics(name, create_relativistic_breit_wigner_with_ff)

```{seealso}
{doc}`/usage/dynamics/custom`
```

And we use the reconfigured {class}`.HelicityAmplitudeBuilder` to generate another {class}`.HelicityModel`, this time with relativistic Breit-Wigner functions and form factors:

In [ ]:
model = model_builder.formulate()

## Export

There is no special export function to export an {class}`.HelicityModel`. However, we can just use the built-in {mod}`pickle` method to write the model to disk:

In [ ]:
import pickle

with open("helicity_model.pickle", "wb") as stream:
    pickle.dump(model, stream)

This model will be imported again in {doc}`/usage/interactive`.

## Visualize

### Mathematical formula

It's possible to view the complete amplitude model as an expression. This would, however, clog the screen here, so we instead just view the formula of one of its {attr}`~.HelicityModel.components`:

In [ ]:
some_amplitude = list(model.components.values())[0]
some_amplitude.doit().n(2)

```{note}
We use {meth}`~sympy.core.basic.Basic.doit` to evaluate the Wigner-$D$ ({meth}`Rotation.D <sympy.physics.quantum.spin.Rotation.D>`) and Clebsch-Gordan ({class}`~sympy.physics.quantum.cg.CG`) functions in the full expression.
```

The {attr}`.HelicityModel.parameter_defaults` attribute can be used to substitute all parameters with suggested values:

In [ ]:
some_amplitude.doit().subs(model.parameter_defaults).n(2)

:::{tip}

To view the full expression for the amplitude model without crashing Jupyter Lab, install [`jupyterlab-katex`](https://pypi.org/project/jupyterlab-katex).

:::

### Plotting

In this case ($J/\psi \to \gamma f_0, f_0 \to \pi^0\pi^0$) _without dynamics_, the total intensity is only dependent on the $\theta$ angle of $\gamma$ in the center of mass frame (see {func}`.get_helicity_angle_label`):

In [ ]:
no_dynamics = model_no_dynamics.expression.doit()
no_dynamics_substituted = no_dynamics.subs(model.parameter_defaults)
no_dynamics_substituted

In [ ]:
assert len(no_dynamics_substituted.free_symbols) == 1

In [ ]:
import sympy as sp

theta = next(iter(no_dynamics_substituted.free_symbols))
sp.plot(
    no_dynamics_substituted,
    (theta, 0, sp.pi),
    axis_center=(0, 0),
    ylabel="$I$",
    ylim=(0, 16),
);

For this decay channel, the amplitude model is built up of four components:

In [ ]:
no_dynamics.subs(zip(no_dynamics.args, sp.symbols("I_{:4}")))

This can be nicely visualized as follows:

In [ ]:
import sympy as sp

plots = []
colors = ["red", "blue", "green", "purple"]

total = 0
for i, intensity in enumerate(no_dynamics.args):
    total += intensity.subs(model.parameter_defaults).doit()
    plots.append(
        sp.plot(
            total,
            (theta, 0, sp.pi),
            axis_center=(0, 0),
            ylabel="$I$",
            ylim=(0, 16),
            line_color=colors[i],
            show=False,
            label=f"$I_{i}$",
            legend=True,
        )
    )
for i in range(1, 4):
    plots[0].extend(plots[i])
plots[0].show()

## Plot the model

```{tip}
See {doc}`/usage/interactive` for a much more didactic way to visualize the model!
```

In the model _with dynamics_, we have several free symbols, such as the mass and width of the resonances. For the fitting package these will be considered **parameters** that are to be optimized and (kinematic) **variables** that represent the data set. Examples of parameters are mass ($m_\text{particle}$) and width ($\Gamma_\text{particle}$) of the resonances and certain amplitude coefficients ($C$). Examples of kinematic variables are the helicity angles $\theta$ and $\phi$ and the invariant masses ($m_{ij...}$).

In [ ]:
sorted(model.expression.free_symbols, key=lambda s: s.name)

Let's say we want to plot the amplitude model with respect to $m_{3+4}$. We would have to substitute all other free symbols with some value.

First, we can use {attr}`.HelicityModel.parameter_defaults` to substitute the parameters with suggested values:

In [ ]:
suggested_expression = model.expression.subs(model.parameter_defaults)
suggested_expression.free_symbols

Ideally, we would now 'integrate out' the helicity angles. Here, we however just set these angles to $0$, as computing the integral would take quite some time:

In [ ]:
angle = 0
angle_substitutions = {
    s: angle
    for s in suggested_expression.free_symbols
    if s.name.startswith("phi") or s.name.startswith("theta")
}
evaluated_angle_intensity = suggested_expression.subs(angle_substitutions)
evaluated_angle_intensity.free_symbols

By now we are only left with the masses of the final state particles ($m_1$ and $m_2$), since they appear as symbols in the {func}`.relativistic_breit_wigner_with_ff`. Final state particles 3 and 4 are the $\pi^0$'s, so we can just substitute them with their masses.

In [ ]:
from qrules import load_pdg

pi0 = load_pdg()["pi0"]
plotted_intensity = evaluated_angle_intensity.doit().subs(
    {
        sp.Symbol("m_1", real=True): pi0.mass,
        sp.Symbol("m_2", real=True): pi0.mass,
    },
    simultaneous=True,
)

```{tip}
Use {meth}`~sympy.core.basic.Basic.subs` with `simultaneous=True`, as that avoids a bug later on when plotting with {mod}`matplotlib.pyplot`.
```

That's it! Now we are only left with the invariant mass $m_{3+4}$ of the two pions:

In [ ]:
assert len(plotted_intensity.free_symbols) == 1
m = next(iter(plotted_intensity.free_symbols))
m

...and we can plot it with with {func}`sympy.plot <sympy.plotting.plot.plot>`:

In [ ]:
sp.plot(
    plotted_intensity,
    (m, 2 * pi0.mass, 2.5),
    axis_center=(2 * pi0.mass, 0),
    xlabel=fR"$m(\pi^{0}\pi^{0})$",
    ylabel="$I$",
    backend="matplotlib",
);

The expression itself looks like this (after some rounding of the {class}`float` values in this expression using {doc}`tree traversal <sympy:tutorial/manipulation>`):

In [ ]:
from sympy import preorder_traversal


def round_nested(expression: sp.Expr, n_decimals: int = 2) -> sp.Expr:
    for node in preorder_traversal(expression):
        if node.free_symbols:
            continue
        if isinstance(node, (float, sp.Float)):
            expression = expression.subs(node, round(node, n_decimals))
        if isinstance(node, sp.Pow) and node.args[1] == 1 / 2:
            expression = expression.subs(node, round(node.n(), n_decimals))
    return expression

In [ ]:
rounded = round_nested(plotted_intensity)
round_nested(rounded)